In [1]:
# get meshes from https://github.com/gpeyre/2015-SIGGRAPH-convolutional-ot
# install libigl: https://libigl.github.io/libigl-python-bindings/
# install meshplot: https://anaconda.org/conda-forge/meshplot (https://skoch9.github.io/meshplot/tutorial/)
# get meshes from https://github.com/libigl/libigl-tutorial-data

In [2]:
import utils_3d
from utils_3d import *
import meshplot as mp
import matplotlib.pyplot as plt

import torchgeometry as tgm
import torch
import torch.nn as nn
import igl
import numpy as np

In [3]:
def draw_box(p):
    v_box = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1],
                  [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
    edges = np.array([
        [0, 1], [0, 2], [1,3], [3, 2], [0, 4], [4, 5], [5, 1], [4, 6], [6, 7], [7, 5], [7, 3], [6, 2]
    ])
    p.add_points(v_box, shading={"point_color": "green", "point_size": 0.1})
    p.add_edges(v_box, edges, shading={"line_color": "green"})

def visualize(cube, p=None, color='red', point_size=0.03):
    voxels = torch.stack(torch.where(cube[0])).T.numpy()
    if p is None:
        p = mp.plot(voxels/SIZE, c=color, shading={"point_size": point_size})
    else:
        p.add_points(voxels/SIZE, c=color, shading={"point_size": point_size})
    return p

In [5]:
def get_mesh(name, res, obj_type):
    path = f"/mnt/nfs/home/saachij/data/libigl-tutorial-data/{name}" 
    v, f, triangles = utils_3d.read_offset_and_normalize_custom(path, obj_type)
    cpu_voxels, sparse_voxels = utils_3d.get_voxelization(triangles, res)
    sparse_voxels = sparse_voxels.to_dense().unsqueeze(0)
    return v, f, cpu_voxels, sparse_voxels


In [6]:
RES = 0.05

v_1, f_1, cpu_voxels_1, sparse_voxels_1 = get_mesh('armadillo.obj', RES, 'obj') 
v_2, f_2, cpu_voxels_2, sparse_voxels_2 = get_mesh('elephant.obj', RES, "obj")
SIZE = sparse_voxels_1.shape[-1]

RuntimeError: CUDA out of memory. Tried to allocate 11.60 GiB (GPU 0; 31.75 GiB total capacity; 18.05 GiB already allocated; 11.04 GiB free; 19.36 GiB reserved in total by PyTorch)

In [ ]:
p = mp.plot(v_1.cpu().numpy(), f_1.cpu().numpy())
draw_box(p)

In [7]:
p = visualize(sparse_voxels_1)
mask = torch.zeros_like(sparse_voxels_1)
v1_matched_points = [
    [2, 17, 15], # left arm
    [10, 15, 16], # nose
    [17, 17, 17], # right arm
    [6, 1, 9], # left toe
    [16, 1, 9] # right toe
]
for point in v1_matched_points:
    print(point, mask.shape)
    mask[0][point[0], point[1], point[2]] = 1
visualize(mask, p, "blue", point_size=0.1)
draw_box(p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999999…

[2, 17, 15] torch.Size([1, 20, 20, 20])
[10, 15, 16] torch.Size([1, 20, 20, 20])
[17, 17, 17] torch.Size([1, 20, 20, 20])
[6, 1, 9] torch.Size([1, 20, 20, 20])
[16, 1, 9] torch.Size([1, 20, 20, 20])


In [6]:
p = mp.plot(v_2.cpu().numpy(), f_2.cpu().numpy())
draw_box(p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999999…

In [8]:
p = visualize(sparse_voxels_2)

mask = torch.zeros_like(sparse_voxels_2)
v2_matched_points = [
    [4, 7, 10], # left arm
    [10, 9, 14], # nose
    [16, 7, 10], # right arm
    [8, 1, 9], # left leg
    [12, 1, 9] # right left
]
for point in v2_matched_points:
    print(point, mask.shape)
    mask[0][point[0], point[1], point[2]] = 1
visualize(mask, p, "blue", point_size=0.1)
draw_box(p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999999…

[4, 7, 10] torch.Size([1, 20, 20, 20])
[10, 9, 14] torch.Size([1, 20, 20, 20])
[16, 7, 10] torch.Size([1, 20, 20, 20])
[8, 1, 9] torch.Size([1, 20, 20, 20])
[12, 1, 9] torch.Size([1, 20, 20, 20])


In [12]:
assert len(v1_matched_points) == len(v2_matched_points)
matched_points = []
for i in range(len(v1_matched_points)):
    matched_points.append([
        v1_matched_points[i], v2_matched_points[i]
    ])
matched_points = torch.tensor(matched_points)
print(matched_points)

tensor([[[ 2, 17, 15],
         [ 4,  7, 10]],

        [[10, 15, 16],
         [10,  9, 14]],

        [[17, 17, 17],
         [16,  7, 10]],

        [[ 6,  1,  9],
         [ 8,  1,  9]],

        [[16,  1,  9],
         [12,  1,  9]]])


In [132]:
keypoint_args = { #0.5, 10
    'alpha': 0,
    'gamma_kp': 0.5,
    'R': 11,
    'matched_points': matched_points.double(),
    'gamma': 0.1, # unused in actual class
    'iters': 20 # unused in actual class
}

# keypoint_args = { #0.5, 10
#     'alpha': 1,
#     'gamma_kp': 0.5,
#     'R': 10,
#     'matched_points': matched_points.double(),
#     'gamma': 0.1, # unused in actual class
#     'iters': 20 # unused in actual class
# }


cwd = ConvolutionalWasserstein3D(1, SIZE, keypoint_args['gamma'], keypoint_args)
inp_1 = sparse_voxels_1 + 1e-2
inp_1 = inp_1/inp_1.sum()
inp_2 = sparse_voxels_2 + 1e-2
inp_2 = inp_2/inp_2.sum()
out, w, v = cwd.sinkhorn(inp_1, inp_2, keypoint_args['iters'], return_plan=True)
plan = lambda x: v*cwd.kernel.forward(w*x)
planT = lambda x: w*cwd.kernel.forward_transpose(v*x)

gamma 0.1 gamma_kp 0.5 alpha 0 R 11 matched_points tensor([[[ 2., 17., 15.],
         [ 4.,  7., 10.]],

        [[10., 15., 16.],
         [10.,  9., 14.]],

        [[17., 17., 17.],
         [16.,  7., 10.]],

        [[ 6.,  1.,  9.],
         [ 8.,  1.,  9.]],

        [[16.,  1.,  9.],
         [12.,  1.,  9.]]], dtype=torch.float64)
kernel tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
0 tensor([1.6155e-05, 1.6155e-05, 1.6155e-05, 1.6155e-05], dtype=torch.float64) tensor([0.7737, 0.7737, 0.7737, 0.7737], dtype=torch.float64)
1 tensor([2.0879e-05, 2.0879e-05, 2.0879e-05, 2.0879e-05], dtype=torch.float64) tensor([0.5987, 0.5987, 0.5987, 0.5987], dtype=torch.float64)
2 tensor([2.6984e-05, 2.6984e-05, 2.6984e-05, 2.6984e-05], dtype=torch.float64) tensor([0.4632, 0.4632, 0.4632, 0.4632], dtype=torch.float64)
3 tensor([3.4874e-05, 3.4874e-05, 3.4874e-05, 3.4874e-05], dtype=torch.float64) tensor([0.3584, 0.3584, 0.3584, 0.3584], dtype=torch.float64)
4 tensor([4.5072e-0

In [133]:
v1_coords = torch.stack(torch.where(sparse_voxels_1[0])).T
mask = torch.zeros_like(w)
# kp = torch.tensor([17, 17, 17])# , radius 8
# kp = torch.tensor([6, 1, 9]) # radius 7
kp =  torch.tensor([30, 17, 17])
for i in range(v1_coords.shape[0]):
    coords = v1_coords[i]
    if torch.linalg.norm(coords.float() - kp.float()) >=18:
        continue
    mask[:, :, coords[0], coords[1], coords[2]] = 1
visualize(mask[0])


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7999999…

In [134]:
dist = planT(mask)[0, 0]
dist = dist/dist.sum()
thresh = 2e-3
cube = torch.ones_like(dist)
voxels = torch.stack(torch.where(cube)).T.numpy()
# p = mp.plot(voxels/SIZE, c='black', shading={"point_size": 0.02})
p = mp.plot(cpu_voxels_2, c='red', shading={"point_size": 0.04})
# p.add_points(cpu_voxels_1, c='blue', shading={"point_size": 0.04})
mask_voxels = torch.stack(torch.where(mask[0, 0])).T.numpy()
p.add_points(mask_voxels/SIZE, c="gray", shading={"point_shape": "square", "point_size": 0.04})
p.add_points((voxels/SIZE)[dist.flatten() > thresh], c=dist.flatten()[dist.flatten() > thresh].numpy(), shading={"point_shape": "circle", "point_size": 0.07})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999999…

2

In [19]:
cwd = ConvolutionalWasserstein3D(1, SIZE, 0.1)
inp_1 = sparse_voxels_1 + 1e-2
inp_1 = inp_1/inp_1.sum()
inp_2 = sparse_voxels_2 + 1e-2
inp_2 = inp_2/inp_2.sum()
entropic_args = {'factor': 0.3}
out = cwd.wass_barycenter(torch.stack([inp_1, inp_2]).double(),  torch.tensor([0.5, 0.5]), 10, entropic_args=None)
out[out < 1e-4] = 0
p = visualize(out)
draw_box(p)

0


RuntimeError: expected scalar type Double but found Float

In [ ]:
visualize(out)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}